### Setup ###

In [1]:
# Markdown: users_signup.ipynb
import mysql.connector
import pandas as pd
import bcrypt
from datetime import datetime, timedelta

DB_CONFIG = {"host":"localhost","user":"root","password":"40418","database":"aeras"}
def get_conn(): return mysql.connector.connect(**DB_CONFIG)
def now_bd(): return datetime.utcnow() + timedelta(hours=6)

# User ID generator
def gen_userid_from(signup_date_str, phone, sequence):
    d = datetime.strptime(signup_date_str, "%Y-%m-%d").strftime("%Y%m%d")
    return f"USR{d}-{str(sequence).zfill(4)}"

def hash_password(plain_password):
    salt = bcrypt.gensalt()
    return bcrypt.hashpw(plain_password.encode('utf-8'), salt).decode('utf-8')

def verify_password(plain_password, hashed):
    return bcrypt.checkpw(plain_password.encode('utf-8'), hashed.encode('utf-8'))

# Example: add user
def add_user(name, privilege, phone, plain_password, signup_date_str, sequence):
    userid = gen_userid_from(signup_date_str, phone, sequence)
    phash = hash_password(plain_password)
    conn = get_conn()
    now = now_bd()
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO users (userid, name, privilege, flag, ban, password_hash, sign_up, phone, created_at)
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """, (userid, name, privilege, 0, 0, phash, signup_date_str, phone, now))
        conn.commit()
    conn.close()
    return userid

# Example usage:
new_userid = add_user("Abul Karim","underprivileged","01312731509","12345","2025-11-13",1)
print("New user:", new_userid)


In [2]:
# Show users
conn = get_conn()
df = pd.read_sql("SELECT id, userid, name, privilege, sign_up, phone FROM users;", conn)
conn.close()
print(df.head())


Empty DataFrame
Columns: [id, userid, name, privilege, sign_up, phone]
Index: []


C:\Users\dibya\AppData\Local\Temp\ipykernel_5924\2385942848.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT id, userid, name, privilege, sign_up, phone FROM users;", conn)
